# Testing model building with `carveme` and `memote`

Importing the COBRA module to read sbml models created via the `carveme` method. Subsequently we reconstruct a draft of the GSM model of *S. cerevisae* using the found RefSeq ID. Looking at the [reference genome R64](https://www.ncbi.nlm.nih.gov/data-hub/taxonomy/4932/?utm_source=assembly&utm_medium=referral&utm_campaign=KnownItemSensor:taxname), we see that 6464 have been annotated.  

In [1]:
from cobra.io import read_sbml_model

%%capture
!carve --refseq GCF_000146045.2_R64 --output Saccharomyces.xml

print("DONE WITH THE CARVING!")

In [2]:
model = read_sbml_model('../yeast-GEM.xml') # Model initiated in minimal medium + glucose (flux bound = 1.0)

In [3]:
print("GSM model fact sheet:")
print("Number of reactions:", len(model.reactions))
print("Number of metabolites:", len(model.metabolites))
print("Number of genes included in the model:", len(model.genes))
print("% of genes in model compared to the number of coding genes in Saccharomyces cerevisae R64 (6464 genes):",
      round((len(model.genes)/6464)*100, 2))
print("Gene coverage percentage =", round(len(model.genes)/len(model.reactions)*100, 2), "%")

GSM model fact sheet:
Number of reactions: 4058
Number of metabolites: 2742
Number of genes included in the model: 1150
% of genes in model compared to the number of coding genes in Saccharomyces cerevisae R64 (6464 genes): 17.79
Gene coverage percentage = 28.34 %


Checking the model setup - how the metabolites etc. are named/tagged

In [70]:
model.metabolites[:5]

[<Metabolite s_0001[ce] at 0x7ff762d43d60>,
 <Metabolite s_0002[c] at 0x7ff762d43e80>,
 <Metabolite s_0003[e] at 0x7ff762d43fa0>,
 <Metabolite s_0004[ce] at 0x7ff762d43f70>,
 <Metabolite s_0006[m] at 0x7ff762d43f10>]

In [7]:
model.compartments # compartments used

{'ce': 'cell envelope',
 'c': 'cytoplasm',
 'e': 'extracellular',
 'm': 'mitochondrion',
 'n': 'nucleus',
 'p': 'peroxisome',
 'er': 'endoplasmic reticulum',
 'g': 'Golgi',
 'lp': 'lipid particle',
 'v': 'vacuole',
 'erm': 'endoplasmic reticulum membrane',
 'vm': 'vacuolar membrane',
 'gm': 'Golgi membrane',
 'mm': 'mitochondrial membrane'}

In [9]:
model.metabolites.get_by_id('s_0001[ce]') # Again specifying model naming

<Metabolite s_0001[ce] at 0x7ff762d43d60>

In [95]:
# Investigating where in the model, acetyl-CoA is used
for metabolite in model.metabolites.query('acetyl-CoA', 'name'):
    print(metabolite.name)

acetoacetyl-CoA [cytoplasm]
s_0367[c]
acetoacetyl-CoA [mitochondrion]
s_0370[m]
acetyl-CoA [cytoplasm]
s_0373[c]
acetyl-CoA [mitochondrion]
s_0376[m]
acetyl-CoA [nucleus]
s_0377[n]
acetyl-CoA [peroxisome]
s_0378[p]
acetoacetyl-CoA [peroxisome]
s_2914[p]


In [75]:
# Model default objective function (growth rate)
print("Model objective function:\n", model.objective, sep="")

Model objective function:
Maximize
1.0*r_2111 - 1.0*r_2111_reverse_58b69


In [77]:
# Investigating the growth reaction rate
model.reactions.get_by_id('r_2111')

<Reaction r_2111 at 0x7ff77dd9f250>

In [79]:
model.metabolites.get_by_id('s_0450[c]')

<Metabolite s_0450[c] at 0x7ff788f81ee0>

In [28]:
model.optimize()

<Solution 0.084 at 0x7ff762558850>

In [40]:
# Exchange reactions for uptake of nutrients from medium
for r in model.medium.keys():
    print(model.reactions.get_by_id(r).name, "=",model.medium[r])

ammonium exchange = 1000.0
D-glucose exchange = 1.0
H+ exchange = 1000.0
iron(2+) exchange = 1000.0
oxygen exchange = 1000.0
phosphate exchange = 1000.0
potassium exchange = 1000.0
sodium exchange = 1000.0
sulphate exchange = 1000.0
water exchange = 1000.0
chloride exchange = 1000.0
Cu2(+) exchange = 1000.0
Mn(2+) exchange = 1000.0
Zn(2+) exchange = 1000.0
Mg(2+) exchange = 1000.0
Ca(2+) exchange = 1000.0


In [41]:
model.medium

{'r_1654': 1000.0,
 'r_1714': 1.0,
 'r_1832': 1000.0,
 'r_1861': 1000.0,
 'r_1992': 1000.0,
 'r_2005': 1000.0,
 'r_2020': 1000.0,
 'r_2049': 1000.0,
 'r_2060': 1000.0,
 'r_2100': 1000.0,
 'r_4593': 1000.0,
 'r_4594': 1000.0,
 'r_4595': 1000.0,
 'r_4596': 1000.0,
 'r_4597': 1000.0,
 'r_4600': 1000.0}

In [44]:
model.reactions.r_1714

<Reaction r_1714 at 0x7ff77def9760>

In [54]:
model.metabolites.get_by_id('s_0565[e]')

<Metabolite s_0565[e] at 0x7ff788f42220>

In [82]:
# Experimentation with glucose concentration --> reach maximum growth rate at conc. ca. 575 
medium = model.medium
min_gluc = model.optimize()
print("mu_min_gluc =", min_gluc.fluxes['r_2111'])
with model:
    medium['r_1714'] = 575  # Glucose exchange reaciton
    model.medium = medium
    solution = model.optimize()
    print("mu_max_succ =",solution.fluxes['r_2111'])

mu_min_gluc = 0.08374770604140971
mu_max_succ = 19.816830259464087


In [100]:
for reaction in model.metabolites.get_by_id('s_0367[c]').reactions:
    reaction
    #print(reaction.name, reaction, sep = " | ")

In [101]:
model.metabolites.get_by_id('s_0367[c]').reactions

frozenset({<Reaction r_0103 at 0x7ff788950670>,
           <Reaction r_0559 at 0x7ff789d5ddf0>,
           <Reaction r_4396 at 0x7ff762ab0df0>})